In [21]:
import sys
import os
package_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, package_root)

In [22]:
from image_acquisition.image_sensor import *
from image_acquisition.playback_images import *
from image_acquisition.video_file_writer import *
#from image_acquisition.ueye_python_driver import UEyePythonDriver

from feature_computation.object_tracking import *
from feature_computation.object_detection_and_tracking import *
from feature_computation.visualization import *

import matplotlib as plt

In [23]:
# Load the ONNX model

output_path = "../../../../PRISM/Daten/makerspace_20241126/output/object_tracking_20241127.mp4"
raw_output_path =  "../../../../PRISM/Daten/makerspace_20241126/output/object_tracking_raw_20241127.mp4"


onnx_model_path = "../../../../PRISM/Daten/makerspace_20241126/export/onnx/simplified-model.onnx"
video_path = "C:/Users/xfebloj/Videos/Captures/soldering_recordings/Stefan/rec_1.avi"

trackers = {
   "pad": MultiObjectTracker(class_ids=[0,1,4,5], disappearance_threshold=-1, ratio_threshold=0.7),
   "cabel": MultiObjectTracker(class_ids=2, disappearance_threshold=50, ratio_threshold=0.7),
   "soldering_iron": KalmanFilterTracker(class_ids=3, ratio_threshold=0.7)
}

In [24]:

visualizer = ObjectTrackerVisualization(trackers)

In [25]:
try:
    with VideoFileImageSensor(video_path=video_path) as img_src:
    #with UEyePythonDriver() as img_src:
        with video_file_writer_matching_image_sensor(img_src, output_path) as writer:
            with onnx_network_and_trackers(onnx_model_path, trackers.values()) as network_and_trackers:

                def callback(frame:np.ndarray):
                    network_and_trackers.update_trackers(frame)
                    disp_img = frame.copy()
                    visualizer.draw_visualization_on_frame(disp_img)

                    cv2.imshow("object tracking", disp_img)

                    writer.write(disp_img)

                    

                    # Check if 'q' is pressed to quit
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        raise NoMoreImages


                run_playback(img_src, callback)
finally:
    cv2.destroyAllWindows()

onnx_session profile:  
